![Clarify Logo](https://global-uploads.webflow.com/5e81e464dad44d3a9a32d1f4/5ed10fc3f1ff8467f4466786_logo.svg)

## Getting started
This notebook will show you how to get credentials, authentication token, and read data directly from Clarify. For additional information and other integration and data science tools [check out our docs](https://docs.clarify.us/docs/welcome)!

### Getting credential

To connect with your Clarify account you need to download your credentials from the admin panel in Clarify. 

1. Access the admin panel you need to click on your organization(located on the top right corner) and go to the integrations menu.
![Admin panel](https://files.readme.io/4a58e3d-Screenshot_2021-03-24_at_16.16.18.png) 
2. Click the integration containing your signal and download the `clarify-credentials.json` file.
![Image of credentials](https://files.readme.io/e2494d1-fe9bd7f-download-credentials.png)
3. The final step is to upload the file to this workspace.


In [ ]:
# install dependencies
!pip install requests_oauthlib "jsonrpcclient[requests]"

In [ ]:
# import credentials
import json
f = open('clarify-credentials.json',)
credentials = json.load(f)
f.close()

In [ ]:
# get access token
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

session = OAuth2Session(client = BackendApplicationClient(client_id=credentials["credentials"]["clientId"]))

try:
    session.fetch_token(
        body=f'audience={credentials["apiUrl"]}', 
        token_url="https://login.clarify.us/oauth/token", 
        client_secret=credentials["credentials"]["clientSecret"]
    )
except:
    raise Exception('failed to get access token: {}'.format(format_exc()))

token = session.access_token

# NB! Does not produce valid token for data retrieval until Public API is finished.

In [ ]:
# id of item (found in item view)
item_id = "" 

In [ ]:
# get data 
from jsonrpcclient.clients.http_client import HTTPClient
from jsonrpcclient.requests import Request

url_endpoint = "https://clarifyapp.clarify.us/api/timeseries/rpc" # api endpoint
method = "timeseries.Floats" # rpc method call

client = HTTPClient(url_endpoint)
headers = {
    "content-type": "application/json",
    "authorization": "Bearer " + token
}

client.session.headers.update(headers)

rpc_text = Request(
    method,
    query = {
        "id": item_id
    },
    window = {
        "from": "2017-01-20T00:00:00Z",
        "to": "2021-12-21T00:00:00Z"
    }
)

res = client.send(rpc_text)
result = res.data.result

In [ ]:
import pandas as pd
import numpy as np

# reformat data
data = []
for section in result["sections"]:
    data += section

ts = np.array(data)
time, value = pd.to_datetime(ts[:,0]), pd.to_numeric(ts[:,1])
series = pd.Series(value, index=time)
print(series.head(5))
series.plot();